In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
def findUniqueVehicleIdInDataFrame(dataframe):
    uniqueVehicleId = []
    for idx, row in dataframe.loc[:].iterrows():
        if row['VehicleId'] not in uniqueVehicleId:
            uniqueVehicleId.append(row['VehicleId'])

    return uniqueVehicleId

In [10]:
dataframe = pd.read_csv('data.csv')
print("Number of rows and columns:", dataframe.shape)

uniqueVehicleId = findUniqueVehicleIdInDataFrame(dataframe)

Number of rows and columns: (158606, 10)


In [11]:
frames = []
for vehicleId in uniqueVehicleId:
    temporaryDataframe = str(vehicleId)
    temporaryDataframe = dataframe.loc[dataframe['VehicleId'] == vehicleId]
    frames.append(temporaryDataframe)
df = pd.concat(frames)

In [15]:
df.head(5)

,TimeStamp,DistanceToStopBar,VehicleId,VehicleType,Speed,Heading,LocationOnMap,LaneId,ApproachId,SignalGroup
0,1623199945,345,68,Car,11.18,90.14,inbound,14,5,2
1,1623199945,344,68,Car,11.20,90.14,inbound,14,5,2
2,1623199945,343,68,Car,11.21,90.14,inbound,14,5,2
3,1623199945,342,68,Car,11.23,90.14,inbound,14,5,2
4,1623199945,341,68,Car,11.24,90.14,inbound,14,5,2


In [16]:
df['VehicleType'] = df['VehicleType'].replace(['Transit'],6)
df['VehicleType'] = df['VehicleType'].replace(['Truck'],9)
df['VehicleType'] = df['VehicleType'].replace(['EmergencyVehicle'],2)
df['VehicleType'] = df['VehicleType'].replace(['Car'],4)
df.head(5)

,TimeStamp,DistanceToStopBar,VehicleId,VehicleType,Speed,Heading,LocationOnMap,LaneId,ApproachId,SignalGroup
0,1623199945,345,68,4,11.18,90.14,inbound,14,5,2
1,1623199945,344,68,4,11.20,90.14,inbound,14,5,2
2,1623199945,343,68,4,11.21,90.14,inbound,14,5,2
3,1623199945,342,68,4,11.23,90.14,inbound,14,5,2
4,1623199945,341,68,4,11.24,90.14,inbound,14,5,2


In [17]:
df['LocationOnMap'] = df['LocationOnMap'].replace(['inbound'],2)
df['LocationOnMap'] = df['LocationOnMap'].replace(['outbound'],4)
df['LocationOnMap'] = df['LocationOnMap'].replace(['outBound'],4)
df['LocationOnMap'] = df['LocationOnMap'].replace(['insideIntersectionBox'],1)

df.head(5)

,TimeStamp,DistanceToStopBar,VehicleId,VehicleType,Speed,Heading,LocationOnMap,LaneId,ApproachId,SignalGroup
0,1623199945,345,68,4,11.18,90.14,2,14,5,2
1,1623199945,344,68,4,11.20,90.14,2,14,5,2
2,1623199945,343,68,4,11.21,90.14,2,14,5,2
3,1623199945,342,68,4,11.23,90.14,2,14,5,2
4,1623199945,341,68,4,11.24,90.14,2,14,5,2


In [18]:
tranining_setsize = int(df.shape[0]*0.8)
print(tranining_setsize)

126884


In [27]:
df_for_training = df.iloc[:tranining_setsize, 1:dataframe.shape[1]]
df_for_testing = df.iloc[tranining_setsize:, 1:dataframe.shape[1]]

In [25]:
df_for_training.head()

,DistanceToStopBar,VehicleId,VehicleType,Speed,Heading,LocationOnMap,LaneId,ApproachId,SignalGroup
0,345,68,4,11.18,90.14,2,14,5,2
1,344,68,4,11.20,90.14,2,14,5,2
2,343,68,4,11.21,90.14,2,14,5,2
3,342,68,4,11.23,90.14,2,14,5,2
4,341,68,4,11.24,90.14,2,14,5,2


In [72]:
trainX = []
trainY = []

In [73]:
# def split_series(datframe, n_past, n_future):
#   #
#   # n_past ==> no of past observations
#   #
#   # n_future ==> no of future observations 
#   #
#     X, y = list(), list()
#     for vehicleId in uniqueVehicleId:
#         temporaryDataframe = str(vehicleId)
#         temporaryDataframe = datframe.loc[datframe['VehicleId'] == vehicleId]
        
#         sc = MinMaxScaler(feature_range = (0, 1))
#         temporaryDataframe_scaled = sc.fit_transform(temporaryDataframe)

#         for window_start in range(len(temporaryDataframe_scaled)):
#             past_end = window_start + n_past
#             future_end = past_end + n_future
#             if future_end > len(temporaryDataframe_scaled):
#                 break
#         # slicing the past and future parts of the window
#             past, future = temporaryDataframe_scaled[window_start:past_end, :], temporaryDataframe_scaled[past_end:future_end, :]
#             X.append(past)
#             y.append(future)
#     return np.array(X), np.array(y)

In [74]:
n_past = 50
n_future = 1 
n_features = 9


In [75]:
uniqueVehicleId = findUniqueVehicleIdInDataFrame(df_for_training)

In [76]:
for vehicleId in uniqueVehicleId:
    #x = len(df[df['VehicleId'] ==  vehicleId])

    temporaryDataframe = str(vehicleId)
    temporaryDataframe = df_for_training.loc[df_for_training['VehicleId'] == vehicleId]
    
    sc = MinMaxScaler(feature_range = (0, 1))
    temporaryDataframe_scaled = sc.fit_transform(temporaryDataframe)

    for i in range (n_past, len(temporaryDataframe_scaled) - n_future + 1):
        trainX.append(temporaryDataframe_scaled[i - n_past:i, 0:temporaryDataframe_scaled.shape[1]])
        trainY.append(temporaryDataframe_scaled[i + n_future -1:i +n_future, 0])

In [77]:
trainX, trainY = np.array(trainX), np.array(trainY)

In [78]:
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (115384, 50, 9).
trainY shape == (115384, 1).


In [60]:
trainX = trainX.reshape((trainX.shape[0], trainX.shape[1],n_features))

In [61]:
# trainY = trainY.reshape((trainY.shape[0], trainY.shape[1],n_features))
trainY = trainY.reshape((trainY.shape[0], trainY.shape[1],0))

ValueError: cannot reshape array of size 1121256 into shape (124584,1,0)

In [62]:
trainY

array([[[0.96811594, 0.        , 0.        , ..., 0.93333333,
         1.        , 0.4       ]],

       [[0.96521739, 0.        , 0.        , ..., 0.93333333,
         1.        , 0.4       ]],

       [[0.96231884, 0.        , 0.        , ..., 0.93333333,
         1.        , 0.4       ]],

       ...,

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]]])

In [9]:
# X_train, y_train = split_series(train.values,n_past, n_future)
# X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
# y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
# X_test, y_test = split_series(test.values,n_past, n_future)
# X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
# y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

NameError: name 'train' is not defined